## Imports

In [11]:
import sklearn
sklearn.__file__


'/home/bruna/.local/lib/python3.6/site-packages/sklearn/__init__.py'

In [13]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
import math
import collections
from os import listdir
from os.path import isfile, join
from scipy.spatial.distance import cosine
import time

import numpy as np
import matplotlib.pyplot as plt

# from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
# from sklearn.datasets.samples_generator import make_blobs

from pprint import pprint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

np.random.seed(42)
plt.rcParams['figure.figsize'] = [15, 5]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/bruna/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-c192863e584b>", line 19, in <module>
    from sklearn.metrics.pairwise import pairwise_distances_argmin
ModuleNotFoundError: No module named 'sklearn.metrics'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/bruna/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/bruna/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_c

ModuleNotFoundError: No module named 'sklearn.metrics'

## Construção da Base de Dados

In [ ]:
# PEGA PATH DE TODAS IMAGENS E COLOCA NUM ARRAY
all_paths = []
classes = listdir('./natural_images_100/')

for root, dirs, files in os.walk('./natural_images_100/'):
    for d in dirs:
        for f in listdir(root + d):
            all_paths.append(root + d + '/' + f)
# classes

In [ ]:
# COLOCA TDS IMAGENS EM UM ARRAY DE IMAGENS PIL 224x224
all_images = np.empty(len(all_paths), dtype=object)
for n in range(0, len(all_paths)):
    all_images[n] = image.load_img(all_paths[n], target_size=(224, 224))


In [ ]:
def print_image(all_images_index):
    img = all_images[all_images_index]
    plt.imshow(img)
    plt.show()
    plt.close()

## Criando o modelo

In [ ]:
# Parâmetros que vamos utilizar

# Utilizar modelo pré-treinado no ImageNet
weights = 'imagenet'

# Utilizar a rede sem a camada final de classificação
include_top = False

# Indicamos que queremos um average pooling na última camada
pooling = 'avg'

# Shape de entrada.
input_shape=(224, 224, 3)

In [ ]:
# keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
model = VGG16(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
model.summary()

In [ ]:
# calculating to all features
# SALVA ARRAY DE FEATURES EM UM GRANDE ARRAY PARA TDS FEATURES
all_features = np.empty((len(all_images), model._nested_outputs.shape[1]))
feature_list_np = []
feature_list = []

for i in range(len(all_images)):
    x = image.img_to_array(all_images[i])
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    feature_list_np.append(np.array(features))
    feature_list.append((np.array(features)).flatten())

    for feature in features:
        all_features[i] = feature

In [ ]:
print(np.shape(all_features))
print(np.shape(all_features[0]))
print()
print(np.shape(feature_list))
print(np.shape(feature_list[0]))

In [ ]:
all_features

In [ ]:
feature_list

In [ ]:
# printa features da primeira imagem
len(all_features)
pprint(all_features[0])
for a in all_features:
    print(a)

In [ ]:
# Verificando shape e tipo das features da prmeira imagem
print(all_features[0].shape, features.dtype, '\n')

# Mostrando as features.
pprint(features[0])

# 2 Aplicação de Algoritmos de Agrupamento de Dados

### Partitioning-based

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn_extra.cluster import KMedoids
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

# X = all_features
df = pd.DataFrame(feature_list)
kmedoids = KMedoids(n_clusters=len(classes), random_state=0).fit(df)
print('Inércia do KMedoids: ', kmedoids.inertia_)
# reduced_data = PCA(n_components=len(classes)).fit_transform(feature_list)
# print(dir(reduced_data))

In [ ]:
miniBatchKmeans = MiniBatchKMeans(n_clusters=len(classes), random_state=0).fit(df)
# miniBatchKmeans = miniBatchKmeans.partial_fit(all_features[0:200,:])
# miniBatchKmeans = miniBatchKmeans.partial_fit(all_features[200:400,:])
# miniBatchKmeans.cluster_centers_
prediction = miniBatchKmeans.predict([all_features[0], all_features[722]]) 

print('Inércia do Mini Batch Kmeans: ', miniBatchKmeans.inertia_)
print('Classes: ', classes)
print('Prediction: ', prediction)
# print_image(722)


In [ ]:
print(__doc__)



# #############################################################################
# Generate sample data
np.random.seed(0)

batch_size = 45
# centers = [[1, 1], [-1, -1], [1, -1]]
n_clusters = len(centers)
# X, labels_true = make_blobs(n_samples=3000, centers=centers, cluster_std=0.7)
# X = feature_list
X = all_features
# X = np.array(feature_list)
print(type(X))
# #############################################################################
# Compute clustering with Means

k_means = KMeans(n_clusters=len(classes), random_state=0)
t0 = time.time()
k_means.fit(X)
t_batch = time.time() - t0

# #############################################################################
# Compute clustering with MiniBatchKMeans

mbk = MiniBatchKMeans(n_clusters=len(classes), batch_size=batch_size, random_state=0)
t0 = time.time()
mbk.fit(X)
t_mini_batch = time.time() - t0

# #############################################################################
# Plot result

fig = plt.figure(figsize=(12, 3))
fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
colors = ['black', 'magenta', 'orchid', 'darkorange', 'red', 'yellow', 'green', 'cyan']

# We want to have the same colors for the same cluster from the
# MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per
# closest one.
k_means_cluster_centers = np.sort(k_means.cluster_centers_, axis=0)
mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis=0)
k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)
mbk_means_labels = pairwise_distances_argmin(X, mbk_means_cluster_centers)
order = pairwise_distances_argmin(k_means_cluster_centers, mbk_means_cluster_centers)

# KMeans
ax = fig.add_subplot(1, 1, 1)
for k, col in zip(range(len(classes)), colors):
    my_members = k_means_labels == k
    cluster_center = k_means_cluster_centers[k]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w', markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=6)
ax.set_title('KMeans')
ax.set_xticks(())
ax.set_yticks(())

plt.show()

In [ ]:
print(__doc__)

import time

import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets.samples_generator import make_blobs

# #############################################################################
# Generate sample data
np.random.seed(0)

batch_size = 45
centers = [[1, 1], [-1, -1], [1, -1]]
n_clusters = len(centers)
X, labels_true = make_blobs(n_samples=3000, centers=centers, cluster_std=0.7)

# #############################################################################
# Compute clustering with Means

k_means = KMeans(init='k-means++', n_clusters=3, n_init=10)
t0 = time.time()
k_means.fit(X)
t_batch = time.time() - t0

# #############################################################################
# Compute clustering with MiniBatchKMeans

mbk = MiniBatchKMeans(init='k-means++', n_clusters=3, batch_size=batch_size,
                      n_init=10, max_no_improvement=10, verbose=0)
t0 = time.time()
mbk.fit(X)
t_mini_batch = time.time() - t0

# #############################################################################
# Plot result

fig = plt.figure(figsize=(8, 3))
fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
colors = ['#4EACC5', '#FF9C34', '#4E9A06']

# We want to have the same colors for the same cluster from the
# MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per
# closest one.
k_means_cluster_centers = np.sort(k_means.cluster_centers_, axis=0)
mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis=0)
k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)
mbk_means_labels = pairwise_distances_argmin(X, mbk_means_cluster_centers)
order = pairwise_distances_argmin(k_means_cluster_centers, mbk_means_cluster_centers)


# KMeans
ax = fig.add_subplot(1, 3, 1)
for k, col in zip(range(n_clusters), colors):
    my_members = k_means_labels == k
    cluster_center = k_means_cluster_centers[k]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w', markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=6)
ax.set_title('KMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8,  'train time: %.2fs\ninertia: %f' % (t_batch, k_means.inertia_))

# MiniBatchKMeans
ax = fig.add_subplot(1, 3, 2)
for k, col in zip(range(n_clusters), colors):
    my_members = mbk_means_labels == order[k]
    cluster_center = mbk_means_cluster_centers[order[k]]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w', markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=6)
ax.set_title('MiniBatchKMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8, 'train time: %.2fs\ninertia: %f' % (t_mini_batch, mbk.inertia_))

# Initialise the different array to all False
different = (mbk_means_labels == 4)
ax = fig.add_subplot(1, 3, 3)

for k in range(n_clusters):
    different += ((k_means_labels == k) != (mbk_means_labels == order[k]))

identic = np.logical_not(different)
ax.plot(X[identic, 0], X[identic, 1], 'w', markerfacecolor='#bbbbbb', marker='.')
ax.plot(X[different, 0], X[different, 1], 'w', markerfacecolor='m', marker='.')
ax.set_title('Difference')
ax.set_xticks(())
ax.set_yticks(())

plt.show()

In [ ]:
kmeans = KMeans(n_clusters=len(classes), random_state=0).fit(np.array(np.array(feature_list)))
# dir(kmeans)
print('Inércia do Kmeans: ', kmeans.inertia_)

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.datasets import load_nfl

# X = all_features
X = np.array(feature_list)
km = KMeans(8, random_state=0)
visualizer = SilhouetteVisualizer(km, colors='yellowbrick')

visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
# COMPARA DUAS IMAGENS PARECIDAS PRA VALIDAR A AS FEATURES
distance = cosine(feature_list[0], feature_list[2])
print(distance)
print_image(0)
print_image(2)

In [ ]:
# model = KMeans(n_clusters=k, random_state=random_state, verbose=verbose)
# data_copy = df.copy()
# # data_copy.drop(columns='label', inplace=True)

# kmeans.fit(data_copy)
# data_copy['cluster_labels'] = kmeans.labels_
# _ = sns.scatterplot(x='x', y='y', data=data_copy, hue='cluster_labels', palette='rainbow', legend=False)

# centroids = kmeans.cluster_centers_
# centroids_x = centroids[:,0]
# centroids_y = centroids[:,1]

# _ = plt.scatter(centroids_x, centroids_y, color='black', s=100)


In [ ]:
# kmeans	scipy.cluster.kmeans.kmeans
# Hierarchical Cluster	scipy.cluster.hierarchy.fcluster
# DBSCAN	sklearn.cluster.DBSCAN
# Birch	sklearn.cluster.Birch
# K-Medoids Cluster	
# pyclust.KMedoids(Unknown reliability)

In [ ]:

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = 0.02  # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

plt.figure()
plt.clf()

plt.suptitle("Comparing multiple K-Medoids metrics to K-Means and each other", fontsize=14)


selected_models = [
    (KMedoids(metric="manhattan", n_clusters=len(classes)), "KMedoids (manhattan)"),
    (KMedoids(metric="euclidean", n_clusters=len(classes)), "KMedoids (euclidean)"),
    (KMedoids(metric="cosine", n_clusters=len(classes)), "KMedoids (cosine)"),
    (KMeans(n_clusters=num_classes), "KMeans"),
]

plot_rows = int(np.ceil(len(selected_models) / 2.0))
plot_cols = 2

for i, (model, description) in enumerate(selected_models):
    # Obtain labels for each point in mesh. Use last trained model.
    model.fit(reduced_data)
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.subplot(plot_cols, plot_rows, i + 1)
    plt.imshow(
        Z,
        interpolation="nearest",
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        cmap=plt.cm.Paired,
        aspect="auto",
        origin="lower",
    )

    plt.plot(reduced_data[:, 0], reduced_data[:, 1], "k.", markersize=2, alpha=0.3)
    # Plot the centroids as a white X
    centroids = model.cluster_centers_
    plt.scatter(centroids[:, 0], centroids[:, 1], marker="x", s=169, linewidths=3, color="w",zorder=10)
    plt.title(description)
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xticks(())
    plt.yticks(())

plt.show()

### Hierarchical-based

### Density-based

### Grid-based

### Model-based

# 3 Avaliação do resultado dos agrupamentos de dados

### Interno

### Externo

### Relativo